In [ ]:


%matplotlib widget
%load_ext autoreload
%autoreload 2
from src.cort_processor import *

from src.plotter import *
from src.filters import *
import math
import pickle
import scipy
import matplotlib.pyplot as plt
from  matplotlib.colors import LinearSegmentedColormap
from matplotlib.pyplot import cm
from scipy import signal
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.stats as stats
from matplotlib import colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import tdt 
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, iirnotch, filtfilt, resample, hilbert, welch
from scipy.fftpack import fft,fftfreq,rfft,irfft,ifft
from scipy.io import savemat
from src.phase_decoder_support import *
from src.neural_analysis import *
from src.wiener_filter import *
from src.folder_handler import *
from src.tdt_support import *
from src.decoders import *

#this first check ensures that regardless of the input sequence, the point by point predictions of the wiener filter will remain static 
original_data = test_rates
data_length = original_data.shape[0]
shuf_order = np.arange(data_length)
np.random.shuffle(shuf_order)
shuffled_data = original_data[shuf_order] 

shuff_predicted_sin = predicted_lines(shuffled_data, h_sin)
shuff_predicted_cos = predicted_lines(shuffled_data, h_cos)
shuff_predicted_arctans = arctan_fn(shuff_predicted_sin, shuff_predicted_cos)

unshuf_order = np.zeros_like(shuf_order)
unshuf_order[shuf_order] = np.arange(data_length)
unshuffled_data = shuff_predicted_arctans[unshuf_order]

tst = np.linspace(0, (test_arctans.shape[0]*50)/1000,test_arctans.shape[0])
fig, axs = plt.subplots(1, 1, figsize=(8,10), sharex= True)
axs.set_title('rollie 11/29 knee phase prediction (test_r2 = 0.49)')
axs.plot(tst, unshuffled_data[:,1], c='black', alpha = 0.8)
axs.plot(tst, predicted_arctans[:,1], c='blue', linestyle = '--', alpha = 0.7)
# axs.plot(tst, shuff_predicted_arctans[:,1], c='red', linestyle = '--', alpha = 0.7)

axs.set_ylabel('degree')
axs.set_xlabel('time (s)')

#this second check ensures that regardless of the sequence of training data presented to the decoding algortihm, the resulting decoder matrix is insensitive so long as reformatted rates are tied to their respective angles 
#this will fail, as it should, if the sin and cos used to train are not likewise shuffled 

full_rates, full_angles = rollie1129.stitch_and_format(rollie1129.data['rates'], rollie1129.data['angles'])
print(full_rates.shape)
_, _, _, _, _, _, phase_list = rollie1129.decode_phase()

angle_number = 2

sin_array, cos_array = sine_and_cosine(phase_list)


original_data = full_rates
data_length = original_data.shape[0]
shuf_order = np.arange(data_length)
np.random.shuffle(shuf_order)
shuffled_X = original_data[shuf_order] 

#this will pass
shuffled_sin = sin_array[shuf_order] 
shuffled_cos = cos_array[shuf_order] 
# #this will fail
# shuffled_sin = sin_array
# shuffled_cos = cos_array 


shuf_h_sin, _, _, _, shuf_sin_test_index = decode_kfolds(X=shuffled_X, Y=shuffled_sin, metric_angle=angle_number, vaf_scoring=False)
shuf_h_cos, _, _, _, _ = decode_kfolds(X=shuffled_X, Y=shuffled_cos, metric_angle=angle_number, vaf_scoring=False, forced_test_index = shuf_sin_test_index)
shuff_predicted_sin = predicted_lines(full_rates, shuf_h_sin)
shuff_predicted_cos = predicted_lines(full_rates, shuf_h_cos)
shuff_predicted_arctans = arctan_fn(shuff_predicted_sin, shuff_predicted_cos)

h_sin, r2_sin, test_rates, test_sin, sin_test_index = decode_kfolds(X=full_rates, Y=sin_array, metric_angle=angle_number, vaf_scoring=False)
h_cos, r2_cos, _, test_cos, _ = decode_kfolds(X=full_rates, Y=cos_array, metric_angle=angle_number, vaf_scoring=False, forced_test_index = sin_test_index)
predicted_sin = predicted_lines(full_rates, h_sin)
predicted_cos = predicted_lines(full_rates, h_cos)
predicted_arctans = arctan_fn(predicted_sin, predicted_cos)

tsf = np.linspace(0, (phase_list.shape[0]*50)/1000,phase_list.shape[0])
fig, axs = plt.subplots(2, 1, figsize=(8,10), sharex= True)
# axs.set_title('rollie 11/29 knee phase prediction (test_r2 = 0.49)')
axs[0].plot(tsf, predicted_arctans[:,1], c='black', alpha = 0.8)
axs[1].plot(tsf, shuff_predicted_arctans[:,1], c='blue', linestyle = '--', alpha = 0.7)
# axs.plot(tst, shuff_predicted_arctans[:,1], c='red', linestyle = '--', alpha = 0.7)